In [11]:
from __future__ import absolute_import, division, print_function
from __future__ import absolute_import, division, print_function

import sys
import os

import pandas as pd
import numpy as np
import datetime
from scipy.stats import norm
from sklearn import metrics

# TSA from Statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

#import statsmodels.api.tsa.statespace
from statsmodels import *
from statsmodels.api import *
from statsmodels.graphics.api import qqplot

# Display and Plotting
import matplotlib.pylab as plt
import seaborn as sns

pd.set_option('display.float_format', lambda x: '%.5f' % x) # pandas
np.set_printoptions(precision=5, suppress=True) # numpy

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# seaborn plotting style
sns.set(style='ticks', context='poster')

#################

df = pd.read_csv('D:\data from MnM Samdal raw\Samdal_power_1h_no_outliers.csv', index_col=0, parse_dates=[0])

df.power = df.power.interpolate(method='linear', limit = 1 )
df.speed = df.speed.interpolate(method='linear', limit = 1 )

ts_df = df
n_sample = ts_df.shape[0] 
    

# Create a training sample and testing sample before analyzing the series
n_train=int(0.95*n_sample)+1
n_forecast=n_sample-n_train
#ts_df
ts_train = ts_df.iloc[:n_train]['power']
ts_test = ts_df.iloc[n_train:]['power']
ts_train_speed = ts_df.iloc[:n_train]['speed']
ts_test_speed = ts_df.iloc[n_train:]['speed']



########################

endog = ts_df.ix[:, 'power']
exog = sm.add_constant(ts_df.ix[:, 'speed'])

nobs = endog.shape[0]

#Model Estimation
arima200 = sm.tsa.statespace.SARIMAX(endog, exog=exog, order=(3,0,1))
model_results = arima200.fit()

arima2002 = sm.tsa.statespace.SARIMAX(endog, exog=exog, order=(3,0,1))
res = arima2002.filter(model_results.params)    

###################

# In-sample one-step-ahead predictions
predict = res.get_prediction()
predict_ci = predict.conf_int()

# # Dynamic predictions
# predict_dy = res.get_prediction(dynamic=start_date)
# predict_dy_ci = predict_dy.conf_int()

max_power =np.max(ts_df.power)

#error definition
def mean_absolute_percentage_error_value(y_true, y_pred):
        to_mean = []
        for true, pred in zip(y_true, y_pred):
            if true != 0:
                to_mean.append(np.abs((true - pred) / true))
        return np.mean(to_mean) * 100

def mae( y_true, y_pred):
        return metrics.mean_absolute_error(y_true, y_pred)

def rmse( y_true, y_pred):
        return np.sqrt(metrics.mean_squared_error(y_true, y_pred))

    
ts_df['timestamp_df'] = ts_df.index


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:475: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [4]:
no_steps = np.arange(1,25,1)
start_dates = np.arange(1,3000,150)

list_nMAE_for_horizon = []

for t in no_steps:
    print('time horizon [h]: ', t , '\n')
    
    # convert to int to avoid error
    t=t.item()
    t=int(t)
        
    list_nMAE = []
    list_nRMSE = []
    
    for i in start_dates:

        i= int(i)
        end_date = i+t-1    
        
        # Dynamic predictions
        predict_dy = res.get_prediction(dynamic=i)
        predict_dy_ci = predict_dy.conf_int()

        #conversion of index numbers to dates
        start_date_date_format = ts_df.iloc[i]['timestamp_df'].strftime('%Y-%m-%d %H:%M')
        end_date_date_format = ts_df.iloc[i+t-1]['timestamp_df'].strftime('%Y-%m-%d %H:%M')
        end_date_date_format_index = ts_df.iloc[i+t]['timestamp_df'].strftime('%Y-%m-%d %H:%M')

        predict_steps = res.forecast(steps=t, exog=exog.ix[start_date_date_format:end_date_date_format])   

        new_index=pd.date_range(start_date_date_format, end_date_date_format, freq='H')

        df = pd.DataFrame(data=predict_steps)
        df['new_index'] = new_index
        df = df.set_index('new_index')
      
        predict_steps_new_idx = df

        pedicted_series_one_step = predict.predicted_mean.iloc[i:i+t]
        pedicted_series_dynamic = predict_dy.predicted_mean.iloc[i:i+t]

        ts_index = pedicted_series_dynamic.index.values


        ##################
        # dynamic 

        Predict_no_zeros = []

        for k in pedicted_series_dynamic:
            if k < 0:
                k=0
                Predict_no_zeros.append(k)
            else:
                Predict_no_zeros.append(k)

        pedicted_series_dynamic = Predict_no_zeros


        predicted_dy = pd.Series(pedicted_series_dynamic, index = ts_index)

        #########

        end_iloc = int(i+t)
        actual_results = ts_df.iloc[i:end_iloc]['power']
       
        ######################
        # protection against Nan
        if np.any(np.isnan(actual_results))==True:
            break
        
        #############
        
        # dynamic
        MAE_1h_dy = mae(actual_results, predicted_dy)
        RMSE_1h_dy = rmse(actual_results, predicted_dy)

        nMAE_1h_dy = (MAE_1h_dy/max_power)*100
        nRMSE_1h_dy = (RMSE_1h_dy/max_power)*100
        
        #####################

        list_nMAE.append(nMAE_1h_dy)
        list_nRMSE.append(nRMSE_1h_dy)
        
    avg_nMAE = sum(list_nMAE)/(len(list_nMAE))
    avg_nRMSE = sum(list_nRMSE)/(len(list_nRMSE))
        
    print('avg errors for : \n' ,
    'avg_nMAE = ' , avg_nMAE , 
    '\n avg_nRMSE = ' , avg_nRMSE   ,'\n'      )
        
    list_nMAE_for_horizon.append(avg_nMAE)
        
print('list_nMAE: ', list_nMAE_for_horizon)


time horizon [h]:  1 

avg errors for : 
 avg_nMAE =  2.02893765532 
 avg_nRMSE =  2.02893765532 

time horizon [h]:  2 

avg errors for : 
 avg_nMAE =  3.2428879277 
 avg_nRMSE =  3.53593247255 

time horizon [h]:  3 

avg errors for : 
 avg_nMAE =  3.98186256414 
 avg_nRMSE =  4.37020197066 

time horizon [h]:  4 

avg errors for : 
 avg_nMAE =  5.24013451616 
 avg_nRMSE =  6.41243979412 

time horizon [h]:  5 

avg errors for : 
 avg_nMAE =  5.29300237243 
 avg_nRMSE =  6.29315421383 

time horizon [h]:  6 

avg errors for : 
 avg_nMAE =  5.02265775853 
 avg_nRMSE =  6.01649405098 

time horizon [h]:  7 

avg errors for : 
 avg_nMAE =  5.15512052659 
 avg_nRMSE =  6.15703179537 

time horizon [h]:  8 

avg errors for : 
 avg_nMAE =  7.04334987017 
 avg_nRMSE =  8.7140426114 

time horizon [h]:  9 

avg errors for : 
 avg_nMAE =  6.74525498323 
 avg_nRMSE =  8.41726855321 

time horizon [h]:  10 

avg errors for : 
 avg_nMAE =  6.56698133092 
 avg_nRMSE =  8.15871305303 

time horizo

In [5]:
# plotting 3 examples

# Example 1: 2011-04-20 18:00:00  iloc 5
# Example 2: 2011-08-09 03:00:00  iloc 2654
# Example 3: 2011-10-07 03:00:00  iloc 4070


no_steps = np.arange(24,25,1)
start_dates = [5, 2654, 4070]


list_nMAE_for_horizon = []

indexes = []
actual_series = []    
predicted_series = []


for t in no_steps:
    print('time horizon [h]: ', t , '\n')

    t=t.item()
    t=int(t)
        
    list_nMAE = []
    list_nRMSE = []
    
    
    for i in start_dates:

        i= int(i)
        end_date = i+t-1    
        
        # Dynamic predictions
        predict_dy = res.get_prediction(dynamic=i)
        predict_dy_ci = predict_dy.conf_int()

        #conversion of index numbers to dates
        start_date_date_format = ts_df.iloc[i]['timestamp_df'].strftime('%Y-%m-%d %H:%M')
        end_date_date_format = ts_df.iloc[i+t-1]['timestamp_df'].strftime('%Y-%m-%d %H:%M')
        end_date_date_format_index = ts_df.iloc[i+t]['timestamp_df'].strftime('%Y-%m-%d %H:%M')
        
        # prediction
        predict_steps = res.forecast(steps=t, exog=exog.ix[start_date_date_format:end_date_date_format])   
    
        new_index=pd.date_range(start_date_date_format, end_date_date_format, freq='H')     
        df = pd.DataFrame(data=predict_steps)
        df['new_index'] = new_index
        df = df.set_index('new_index')
        
        predict_steps_new_idx = df

        pedicted_series_one_step = predict.predicted_mean.iloc[i:i+t]
        pedicted_series_dynamic = predict_dy.predicted_mean.iloc[i:i+t]

        ts_index = pedicted_series_dynamic.index.values
                
        indexes.append(ts_index)


        #########
        # one step prediction; remove zeros to make more realistic prediction [no negative power]

        Predict_no_zeros = []

        for j in pedicted_series_one_step:
            if j < 0:
                j=0
                Predict_no_zeros.append(j)
            else:
                Predict_no_zeros.append(j)

        pedicted_series_one_step = Predict_no_zeros

        predicted_one = pd.Series(pedicted_series_one_step, index = ts_index)


        ##################
        # dynamic 

        Predict_no_zeros = []

        for k in pedicted_series_dynamic:
            if k < 0:
                k=0
                Predict_no_zeros.append(k)
            else:
                Predict_no_zeros.append(k)

        pedicted_series_dynamic = Predict_no_zeros

        predicted_dy = pd.Series(pedicted_series_dynamic, index = ts_index)
        print('\n predicted_dy' , predicted_dy)
      
        predicted_series.append(predicted_dy.values)

        #########

        end_iloc = int(i+t)
        
        actual_results = ts_df.iloc[i:end_iloc]['power']
        print('\n actual' , actual_results)
        actual_series.append(actual_results.values)

        ######################
        # protection against Nan
        
        if np.any(np.isnan(actual_results))==True:
            break

        #############
        
        # dynamic
        MAE_1h_dy = mae(actual_results, predicted_dy)
        RMSE_1h_dy = rmse(actual_results, predicted_dy)

        nMAE_1h_dy = (MAE_1h_dy/max_power)*100
        nRMSE_1h_dy = (RMSE_1h_dy/max_power)*100
        
        #####################

        list_nMAE.append(nMAE_1h_dy)
        list_nRMSE.append(nRMSE_1h_dy)
        
    avg_nMAE = sum(list_nMAE)/(len(list_nMAE))
    avg_nRMSE = sum(list_nRMSE)/(len(list_nRMSE))
        
    print('avg errors for : \n' ,
    'avg_nMAE = ' , avg_nMAE , 
    '\n avg_nRMSE = ' , avg_nRMSE   ,'\n'      )
                
    list_nMAE_for_horizon.append(avg_nMAE)
        
print('list_nMAE: ', list_nMAE_for_horizon)



time horizon [h]:  24 


 predicted_dy 2011-04-20 17:00:00   1752.82480
2011-04-20 18:00:00   1544.22825
2011-04-20 19:00:00   1606.70520
2011-04-20 20:00:00   1238.32259
2011-04-20 21:00:00   1093.59097
2011-04-20 22:00:00    932.23121
2011-04-20 23:00:00    729.44502
2011-04-21 00:00:00    449.15056
2011-04-21 01:00:00    582.51041
2011-04-21 02:00:00    549.67160
2011-04-21 03:00:00    569.03127
2011-04-21 04:00:00    825.00654
2011-04-21 05:00:00    651.69008
2011-04-21 06:00:00    709.52797
2011-04-21 07:00:00    739.13973
2011-04-21 08:00:00    829.39954
2011-04-21 09:00:00    679.51243
2011-04-21 10:00:00    485.19700
2011-04-21 11:00:00    345.26150
2011-04-21 12:00:00      0.00000
2011-04-21 13:00:00    141.46163
2011-04-21 14:00:00      0.00000
2011-04-21 15:00:00      0.00000
2011-04-21 16:00:00     91.92408
dtype: float64

 actual 2011-04-20 17:00:00   1568.04243
2011-04-20 18:00:00   1291.88987
2011-04-20 19:00:00   1672.94148
2011-04-20 20:00:00    939.29108
2011-04-20 21

In [6]:
# creating dataframe with results of prediction for plotting

indexes = np.array(indexes)
actual_series_dat = np.array(actual_series)
predicted_series_dat = np.array(predicted_series)

indexes = indexes.reshape(-1,)
actual_series_dat = actual_series_dat.reshape(-1,)
predicted_series_dat =  predicted_series_dat.reshape(-1,)                              
                                                          

data_dict = {'actual':actual_series_dat, 
             'predicted':predicted_series_dat, 
            }


predicted_series_dat.shape
test_data = pd.DataFrame(data_dict, index =indexes)


In [7]:
test_data.to_excel('ARIMA_data_for_plotting_new.xlsx')

In [8]:
# Graph
fig, ax = plt.subplots(figsize=(9,4))
npre = 4
ax.set(title='Forecast of power generation', xlabel='Time', ylabel='power [kW]')

# Plot data points
ts_df.ix[start_plot:, 'power'].plot(ax=ax, style='o', label='Observed')

# Plot predictions
predicted_one.plot(ax=ax, style='r--', label='One-step-ahead forecast')
predicted_dy.plot(ax=ax, style='g', label='Dynamic forecast')

legend = ax.legend(['Observed', 'One-step-ahead forecast', 'Dynamic forecast'], loc='upper right')
plt.ylim(-5,)
plt.show()


NameError: name 'start_plot' is not defined

In [10]:
# rmse

no_steps = np.arange(1,25,1)
start_dates = np.arange(1,3000,150)

list_nMAE_for_horizon = []


for t in no_steps:
    print('time horizon [h]: ', t , '\n')

    t=t.item()
    t=int(t)
        
    #list_nMAE = []
    list_nRMSE = []
    
    for i in start_dates:

        i= int(i)
        end_date = i+t-1    

        
        # Dynamic predictions
        predict_dy = res.get_prediction(dynamic=i)
        predict_dy_ci = predict_dy.conf_int()

        #conversion of index numbers to dates
        start_date_date_format = ts_df.iloc[i]['timestamp_df'].strftime('%Y-%m-%d %H:%M')
        end_date_date_format = ts_df.iloc[i+t-1]['timestamp_df'].strftime('%Y-%m-%d %H:%M')
        end_date_date_format_index = ts_df.iloc[i+t]['timestamp_df'].strftime('%Y-%m-%d %H:%M')
        
        predict_steps = res.forecast(steps=t, exog=exog.ix[start_date_date_format:end_date_date_format])   

        #print('\n predict_steps', predict_steps)
        
        new_index=pd.date_range(start_date_date_format, end_date_date_format, freq='H')

        df = pd.DataFrame(data=predict_steps)
        df['new_index'] = new_index
        df = df.set_index('new_index')

        predict_steps_new_idx = df

        # converting negative values of power to zeroes

        pedicted_series_one_step = predict.predicted_mean.iloc[i:i+t]
        pedicted_series_dynamic = predict_dy.predicted_mean.iloc[i:i+t]

        ts_index = pedicted_series_dynamic.index.values

        ##################
        # dynamic 

        Predict_no_zeros = []

        for k in pedicted_series_dynamic:
            if k < 0:
                k=0
                Predict_no_zeros.append(k)
            else:
                Predict_no_zeros.append(k)

        pedicted_series_dynamic = Predict_no_zeros

        predicted_dy = pd.Series(pedicted_series_dynamic, index = ts_index)

        end_iloc = int(i+t)
        
        actual_results = ts_df.iloc[i:end_iloc]['power']
        
        ######################
        # protection against Nan
        
        if np.any(np.isnan(actual_results))==True:
            break
        
       
        # dynamic
        MAE_1h_dy = mae(actual_results, predicted_dy)
        RMSE_1h_dy = rmse(actual_results, predicted_dy)

        nMAE_1h_dy = (MAE_1h_dy/max_power)*100
        nRMSE_1h_dy = (RMSE_1h_dy/max_power)*100
        
        list_nMAE.append(nMAE_1h_dy)
        list_nRMSE.append(nRMSE_1h_dy)
     
    avg_nMAE = sum(list_nMAE)/(len(list_nMAE))
    avg_nRMSE = sum(list_nRMSE)/(len(list_nRMSE))
        
    print('avg errors for : \n' ,
    'avg_nMAE = ' , avg_nMAE , 
    '\n avg_nRMSE = ' , avg_nRMSE   ,'\n'      )
             
    list_nMAE_for_horizon.append(avg_nMAE)
        
print('list_nMAE: ', list_nMAE_for_horizon)



time horizon [h]:  1 

avg errors for : 
 avg_nMAE =  2.76076432233 
 avg_nRMSE =  2.02893765532 

time horizon [h]:  2 

avg errors for : 
 avg_nMAE =  2.96738872463 
 avg_nRMSE =  3.53593247255 

time horizon [h]:  3 

avg errors for : 
 avg_nMAE =  3.27173087649 
 avg_nRMSE =  4.37020197066 

time horizon [h]:  4 

avg errors for : 
 avg_nMAE =  3.72597787026 
 avg_nRMSE =  6.41243979412 

time horizon [h]:  5 

avg errors for : 
 avg_nMAE =  4.01979496441 
 avg_nRMSE =  6.29315421383 

time horizon [h]:  6 

avg errors for : 
 avg_nMAE =  4.17814172138 
 avg_nRMSE =  6.01649405098 

time horizon [h]:  7 

avg errors for : 
 avg_nMAE =  4.31136610391 
 avg_nRMSE =  6.15703179537 

time horizon [h]:  8 

avg errors for : 
 avg_nMAE =  4.43014800679 
 avg_nRMSE =  8.7140426114 

time horizon [h]:  9 

avg errors for : 
 avg_nMAE =  4.52661079748 
 avg_nRMSE =  8.41726855321 

time horizon [h]:  10 

avg errors for : 
 avg_nMAE =  4.60822561881 
 avg_nRMSE =  8.15871305303 

time horiz